# This is a playground for performing experiments before implementing them in the main code using pyspark for quick access

In [126]:
import pandas as pd
import glob

csv_path = "./NBA shot log 16-17-regular season/Shot data/"

# read all csvs in the directory
df = pd.concat([pd.read_csv(f) for f in glob.glob(csv_path + "*.csv")], ignore_index=True)

# Print the number of rows and columns
print(df.shape)

(210072, 16)


In [127]:
# read player data
df_player = pd.read_csv("./NBA shot log 16-17-regular season/Player Regular 16-17 Stats.csv")



print(df_player.columns)
# remove the columns that are not needed [#Date/Time of Update: 2017-05-09 4:34:01 PM, #Player ID, #Jersey Num, #Birth Date, #Birth City, #Birth Couuntry, #Team ID
#                                         #Team Abbr, #Team City, #Team Name]
df_player = df_player.drop(['#Date/Time of Update: 2017-05-09 4:34:01 PM', '#Player ID', '#Jersey Num', '#Birth City', '#Birth Country', '#Team ID', '#Team Abbr', '#Team City', '#Team Name'], axis=1)
print(df_player.columns)

# combine First Name and Last Name to create a new column called Player Name
df_player['shoot player'] = df_player['#FirstName'] + " " + df_player['#LastName']
df_player = df_player.drop(['#FirstName', '#LastName'], axis=1)
print(df_player.columns)

# connect df_player and df[0]
df_merged = pd.merge(df, df_player, on='shoot player', how='inner')
df_merged.to_csv("output/Standard/Single File/merged.csv", index=False)
print(df_merged.columns)

Index(['#Date/Time of Update: 2017-05-09 4:34:01 PM', '#Player ID',
       '#LastName', '#FirstName', '#Jersey Num', '#Position', '#Height',
       '#Weight', '#Birth Date', '#Age', '#Birth City', '#Birth Country',
       '#Rookie', '#Team ID', '#Team Abbr', '#Team City', '#Team Name',
       '#GamesPlayed', '#Fg2PtAtt', '#Fg2PtMade', '#Fg3PtAtt', '#Fg3PtMade',
       '#FtAtt', '#FtMade'],
      dtype='object')
Index(['#LastName', '#FirstName', '#Position', '#Height', '#Weight',
       '#Birth Date', '#Age', '#Rookie', '#GamesPlayed', '#Fg2PtAtt',
       '#Fg2PtMade', '#Fg3PtAtt', '#Fg3PtMade', '#FtAtt', '#FtMade'],
      dtype='object')
Index(['#Position', '#Height', '#Weight', '#Birth Date', '#Age', '#Rookie',
       '#GamesPlayed', '#Fg2PtAtt', '#Fg2PtMade', '#Fg3PtAtt', '#Fg3PtMade',
       '#FtAtt', '#FtMade', 'shoot player'],
      dtype='object')
Index(['self previous shot', 'player position', 'home game', 'location x',
       'opponent previous shot', 'home team', 'shot type', 

In [128]:
df_merged_test = df_merged
# print unique values in self previous shot
columns_to_print = ['self previous shot', 'player position', 'home game', 'opponent previous shot', 'home team', 'points', 'time from last shot', 'quarter', 'current shot outcome', '#Position',
       '#Height', '#Weight', '#Age', '#Rookie', '#GamesPlayed', '#Fg2PtAtt',
       '#Fg2PtMade', '#Fg3PtAtt', '#Fg3PtMade', '#FtAtt', '#FtMade']

# add a new column Fg2PtPct, Fg3PtPct, FtPct
df_merged_test['Fg2PtPct'] = df_merged_test['#Fg2PtMade'] / df_merged_test['#Fg2PtAtt']
df_merged_test['Fg3PtPct'] = df_merged_test['#Fg3PtMade'] / df_merged_test['#Fg3PtAtt']
df_merged_test['FtPct'] = df_merged_test['#FtMade'] / df_merged_test['#FtAtt']

# Drop players with 0 attempts in Fg2PtAtt, Fg3PtAtt, FtAtt
df_merged_test = df_merged_test[df_merged_test['#Fg2PtAtt'] != 0]
df_merged_test = df_merged_test[df_merged_test['#Fg3PtAtt'] != 0]
df_merged_test = df_merged_test[df_merged_test['#FtAtt'] != 0]

# display count of nulls in Fg2PtPct, Fg3PtPct, FtPct
print(df_merged_test['Fg2PtPct'].isnull().sum())
print(df_merged_test['Fg3PtPct'].isnull().sum())
print(df_merged_test['FtPct'].isnull().sum())

# Drop Fg2PtAtt, Fg2PtMade, Fg3PtAtt, Fg3PtMade, FtAtt, FtMade
#df_merged_test = df_merged_test.drop(['#Fg2PtAtt', '#Fg2PtMade', '#Fg3PtAtt', '#Fg3PtMade', '#FtAtt', '#FtMade'], axis=1)
# Drop irrelevant columns
df_merged_test = df_merged_test.drop(['date','#Birth Date', 'away team', 'home team'], axis=1)

# Convert height from feet-inches to inches
y = df_merged_test['#Height'].head(1)
# replace #Height ''6'5"'' with 196
df_merged_test['#Height'] = df_merged_test['#Height'].replace("6'5\"", "196")
# height map
height_map = {'5\'4\"': 64, '5\'9\"': 69, '5\'10\"': 70, '5\'11\"': 71,
       '6\'0\"': 72, '6\'1\"': 73, '6\'2\"': 74, '6\'3\"': 75, '6\'4\"': 76, '6\'5\"': 77, '6\'6\"': 78, '6\'7\"': 79, '6\'8\"': 80,
       '6\'9\"': 81, '6\'10\"': 82, '6\'11\"': 83, '7\'0\"': 84, '7\'1\"': 85, '7\'2\"': 86, '7\'3\"': 87}


# convert height to inches using height_map
df_merged_test['#Height'] = df_merged_test['#Height'].map(height_map)
# replace height nans with the mean
df_merged_test['#Height'] = df_merged_test['#Height'].fillna(round(df_merged_test['#Height'].mean()))
# Normalize height subtracting the min and dividing by the range
df_merged_test['#Height'] = (df_merged_test['#Height'] - df_merged_test['#Height'].min()) / (df_merged_test['#Height'].max() - df_merged_test['#Height'].min())

# Fill age nans with the mean
df_merged_test['#Age'] = df_merged_test['#Age'].fillna(round(df_merged_test['#Age'].mean()))
# Normalize age using z-score
df_merged_test['#Age'] = (df_merged_test['#Age'] - df_merged_test['#Age'].mean()) / df_merged_test['#Age'].std()
# Fill weight nans with the mean
df_merged_test['#Weight'] = df_merged_test['#Weight'].fillna(round(df_merged_test['#Weight'].mean()))
# Normalize weight using z-score
df_merged_test['#Weight'] = (df_merged_test['#Weight'] - df_merged_test['#Weight'].mean()) / df_merged_test['#Weight'].std()

# Convert rookie to 0 and 1
df_merged_test['#Rookie'] = df_merged_test['#Rookie'].map({'N': 0, 'Y': 1})

# drop nans from rows with location x as null
df_merged_test = df_merged_test.dropna(subset=['location x'])

# get columns that contain nans and their counts
for column in df_merged_test.columns:
       if df_merged_test[column].isnull().values.any():
              print(column, df_merged_test[column].isnull().sum())


df_merged_test.to_csv("output/Standard/Single File/merged_test.csv", index=False)

0
0
0
self previous shot 2385
opponent previous shot 1558
time from last shot 9711


In [130]:
for column in ['self previous shot', 'opponent previous shot']:
    print(column + ":", df_merged_test[column].unique())

# print nulls per column in df_player
print(df_merged_test.isnull().sum())

# print first 10 rows with time from last shot as nan
print(df[df['time from last shot'].isnull()].head(10))

self previous shot: [nan 'MISSED' 'SCORED' 'BLOCKED']
opponent previous shot: ['SCORED' 'MISSED' 'BLOCKED' nan]
self previous shot        2385
player position              0
home game                    0
location x                   0
opponent previous shot    1558
shot type                    0
points                       0
location y                   0
time                         0
shoot player                 0
time from last shot       9711
quarter                      0
current shot outcome         0
#Position                    0
#Height                      0
#Weight                      0
#Age                         0
#Rookie                      0
#GamesPlayed                 0
#Fg2PtAtt                    0
#Fg2PtMade                   0
#Fg3PtAtt                    0
#Fg3PtMade                   0
#FtAtt                       0
#FtMade                      0
Fg2PtPct                     0
Fg3PtPct                     0
FtPct                        0
dtype: int64
    sel